In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from vectorstore import connect_to_db
from openai import OpenAI
import json 
import weaviate
from weaviate.collections.classes.filters import Filter
from weaviate.classes.aggregate import GroupByAggregate


# Historique d'utilisation des clusters (pour la pénalisation)
cluster_usage = defaultdict(int)

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  # Replace with your actual API key
)

model = "mistralai/mistral-7b-instruct:free"

/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/cytech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/cytech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_unique_themes():
    client = connect_to_db()

    # Accéder à la collection 'Chunk'
    collection = client.collections.get("Chunk")

    # Effectuer une requête d'agrégation pour obtenir les thèmes uniques
    response = collection.aggregate.over_all(
        group_by=GroupByAggregate(prop="theme")
    )

    # Extraire les thèmes uniques de la réponse
    unique_themes = [group.grouped_by.value for group in response.groups]

    return unique_themes

In [3]:
# Récupérer les chunks de la base de données avec leurs embeddings
def fetch_chunks_from_weaviate(theme):
    client = connect_to_db()

    # Accéder à la collection 'Chunk'
    collection = client.collections.get("Chunk")

    # Définir le filtre pour rechercher les chunks contenant le thème spécifié
    theme_filter = Filter.by_property("theme").contains_any([theme])


    # Initialiser la liste pour stocker les chunks récupérés
    chunks = []
    offset = 0
    batch_size = 100  # Nombre d'objets à récupérer par requête

    while True:
        # Récupérer un lot de chunks correspondant au filtre
        objects = collection.query.fetch_objects(
            filters=theme_filter,
            include_vector=True,
            limit=batch_size,
            offset=offset
        )

        # Vérifier si des objets ont été retournés
        if not objects.objects:
            break

        # Ajouter les chunks récupérés à la liste
        for obj in objects.objects:
            chunk = obj.properties
            chunk['vector'] = obj.vector
            chunks.append(chunk)

        # Mettre à jour l'offset pour la prochaine itération
        offset += batch_size

    return chunks

In [8]:
def cluster_chunks(chunks, cluster_size=15):
    """ Clustre les chunks en groupes d'environ `cluster_size` chunks. """

    n_clusters = max(1, len(chunks) // cluster_size)  # Déterminer le nombre de clusters
    
    embeddings = np.array([chunk['vector']["default"] for chunk in chunks])  # chunk['vector'] est déjà un tableau NumPy
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings)
    
    clustered_chunks = defaultdict(list)
    for i, label in enumerate(labels):
        clustered_chunks[label].append(chunks[i])

    return clustered_chunks

def select_cluster_with_penalty(clustered_chunks):
    cluster_scores = {}

    for cluster_id, chunks in clustered_chunks.items():
        usage_penalty = cluster_usage[cluster_id] * 0.05  # Plus utilisé = plus pénalisé
        cluster_scores[cluster_id] = len(chunks) - usage_penalty
    
    # Ajouter du bruit aléatoire
    for cluster_id in cluster_scores:
        cluster_scores[cluster_id] += random.uniform(-0.1, 0.1)
    
    selected_cluster = max(cluster_scores, key=cluster_scores.get)
    cluster_usage[selected_cluster] += 1  # Mise à jour de la pénalité

    return selected_cluster

# Sélectionner aléatoirement x chunks dans un cluster
def select_chunks_from_cluster(clustered_chunks, selected_cluster, num_chunks=3):
    cluster_chunks = clustered_chunks[selected_cluster]
    return random.sample(cluster_chunks, min(num_chunks, len(cluster_chunks)))

In [12]:
def generate_questions_from_chunks(chunks, model, selected_subtheme):
    """Génère des questions MCQ à partir des chunks sélectionnés en utilisant un LLM."""
    
    combined_texts = "\n\n".join([f"- {chunk['text']}" for chunk in chunks])
        # Collecter les informations source (original_filename, title, section)
     # Collecter les informations source (original_filename, title, section) sous forme de dictionnaire
    sources = {
        "original_filename": chunks[0].get("original_filename", ""),
        "title": chunks[0].get("title", ""),
        "section": chunks[0].get("section", "")
    }
    
    messages = [
        {"role": "system", "content": "You are an expert patent law professor, known for your clear and direct teaching style. I am a patent law student. Your task is to generate multiple-choice questions (MCQs) to evaluate my knowledges."
        "You MUST Adopt an informal tone and address to me directly."


        "I can't see the context of questions, so make sure to provide enough detail for me to understand the questions."
        },
        
        {"role": "user", "content": f"""

    Based on the following context:

    {combined_texts}

    Generate exactly 10 MCQ questions.
    Each question must strictly follow this JSON format:

    [
        {{
            "QuestionText": "Question text",
            "Options": {{
                "A": "Option A",
                "B": "Option B",
                "C": "Option C",
                "D": "Option D"
            }},
            "CorrectAnswer": "A",  # One letter corresponding to the correct answer
            "Explanation": "Explanation of the correct answer"
        }},
        ...
    ]

    Ensure the output is a **valid JSON array** and contains **no additional text**. Also, make sure the **explanation is well detailed**.
        """}
    ]

    # Appel à l'API OpenAI ou autre LLM
    response = client.chat.completions.create(model=model, messages=messages)

    if response.choices:
        try:
            generated_text = response.choices[0].message.content.strip()
            questions_json = json.loads(generated_text)  # Convertir en JSON
        except json.JSONDecodeError:
            print(f"🚨 JSON error with theme {selected_subtheme}: {generated_text}")
            return []

    # Formater les questions
    formatted_questions = []
    for i, question in enumerate(questions_json):
        formatted_questions.append({
            "QuestionNumber": i + 1,
            "QuestionText": question.get("QuestionText", "Question not found"),
            "Options": question.get("Options", {}),
            "CorrectAnswer": question.get("CorrectAnswer", ""),
            "Explanation": question.get("Explanation", ""),
            "Question_type": "MCQ",
            "file": "Generated Questions",
            "Theme": selected_subtheme,
            "Sources": sources
        })
   
    return formatted_questions

In [14]:
# Obtenir la liste des thèmes uniques
unique_themes = get_unique_themes()

# Dictionnaire pour stocker les chunks par thème
chunks_by_theme = {}

# Récupérer les chunks pour chaque thème
for theme in unique_themes:
    chunks = fetch_chunks_from_weaviate(theme)
    #print(f"Nombre de chunks trouvés pour le thème '{theme}' : {len(chunks)}")

    # Clustering des chunks de ce sous-thème
    clusters = cluster_chunks(chunks)

    # Sélectionner un cluster
    selected_cluster = select_cluster_with_penalty(clusters)

    # Prendre x chunks dans ce cluster
    selected_chunks = select_chunks_from_cluster(clusters, selected_cluster, num_chunks=10)
    question = generate_questions_from_chunks(selected_chunks, model, theme)

    print("Thème sélectionné :", theme)
    print("Question générée :", question)
    # Enregistrer le dictionnaire de questions dans un fichier JSON
    with open(f'{theme}.json', 'w', encoding='utf-8') as json_file: 
        json.dump(question, json_file, ensure_ascii=False, indent=4)

Thème sélectionné : Further processing (Rule 135 EPC)
Question générée : [{'QuestionNumber': 1, 'QuestionText': 'What is the effect of a universal successor opponent automatically acquiring party status in proceedings pending at the EPO, according to the EPC 1973?', 'Options': {'A': 'The proceedings are automatically terminated.', 'B': 'The successor opponent can file new objections and participate in the proceedings.', 'C': 'The successor opponent is not allowed to participate in the proceedings.', 'D': 'The proceedings are put on hold until supporting evidence is filed.'}, 'CorrectAnswer': 'B', 'Explanation': 'The universal successor opponent automatically acquires party status in proceedings pending at the EPO, irrespective of whether supporting evidence is filed. This means that the successor opponent can file new objections and participate in the proceedings.', 'Question_type': 'MCQ', 'file': 'Generated Questions', 'Theme': 'Further processing (Rule 135 EPC)', 'Sources': {'origina

/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(
/usr/lib/python3.9/asyncio/selector_events.py:702: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=83 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/usr/lib/python3.9/asyncio/selector_events.py:702: ResourceWarning: unclosed transport 

Thème sélectionné : Grant stage (Rule 71(3) EPC) and post-grant publication
Question générée : [{'QuestionNumber': 1, 'QuestionText': 'Which of the following statements about the European Patent Office (EPO) is correct?', 'Options': {'A': 'The EPO is a legal entity with extensive legal capacity accorded by national law.', 'B': 'The EPO is not a legal entity and does not have any legal capacity.', 'C': 'The EPO is located in Paris, France.', 'D': 'The EPO is not an office and does not perform any activities.'}, 'CorrectAnswer': 'A', 'Explanation': 'The EPO is a legal entity with extensive legal capacity accorded by national law. It can acquire, dispose, and own movable and immovable property, and it can be a party to legal proceedings (Article 2 of the EPC).', 'Question_type': 'MCQ', 'file': 'Generated Questions', 'Theme': 'Grant stage (Rule 71(3) EPC) and post-grant publication', 'Sources': {'original_filename': 'case_law_of_the_boards_of_appeal_2022_en', 'title': 'requirement industri

/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(
/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


Thème sélectionné : Preliminary examination and amendments
Question générée : [{'QuestionNumber': 1, 'QuestionText': 'What is the legal status of the first written decision issued by the opposition division in the given context?', 'Options': {'A': 'Legally invalid and not binding on the parties', 'B': 'Legally valid and binding on the parties', 'C': 'Legally invalid but can be challenged in a separate appeal', 'D': 'Legally valid but can be set aside under certain conditions'}, 'CorrectAnswer': 'B', 'Explanation': 'The first written decision is legally valid and binding on the parties because it was notified to them in accordance with the formal requirements of the EPC 1973 (68, 70, 111, 113 EPC 1973). The opposition division is bound by the legal effect of this decision, even if it could be set aside in a second instance appeal (see OJ 1995 324).', 'Question_type': 'MCQ', 'file': 'Generated Questions', 'Theme': 'Preliminary examination and amendments', 'Sources': {'original_filename':

...

Thème sélectionné : Inventive step analysis
Question générée : [{'QuestionNumber': 1, 'QuestionText': "Which of the following best describes the role of the 'closest prior art' in patent law?", 'Options': {'A': 'It is a document that is not related to the claimed invention and has no bearing on the inventive step assessment.', 'B': 'It is a document that discloses the conceived purpose and aiming objective of the claimed invention, and has relevant technical features that require minimum structural modifications.', 'C': 'It is a document that is not necessarily the most similar to the claimed invention, but serves as a starting point for the inventive step assessment.', 'D': 'It is a document that is always the most similar to the claimed invention and is the only document considered in the inventive step assessment.'}, 'CorrectAnswer': 'B', 'Explanation': "The 'closest prior art' is a document that discloses the conceived purpose and aiming objective of the claimed invention, and has 

/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(
/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


Thème sélectionné : Strategies for overcoming lack of unity
Question générée : [{'QuestionNumber': 1, 'QuestionText': 'What is the objective technical problem in patent law?', 'Options': {'A': 'The aim to modify or adapt the closest prior art to provide technical effects.', 'B': 'The aim to find any prior art that could potentially invalidate the patent.', 'C': 'The aim to find the most distant prior art possible, regardless of its relevance.', 'D': 'The aim to find prior art that is not related to the invention at all.'}, 'CorrectAnswer': 'A', 'Explanation': 'The objective technical problem in patent law is defined as the aim to modify or adapt the closest prior art to provide technical effects. This is done to establish the technical problem that the invention solves, which is a crucial step in assessing the inventive step.', 'Question_type': 'MCQ', 'file': 'Generated Questions', 'Theme': 'Strategies for overcoming lack of unity', 'Sources': {'original_filename': '3-en-epc-guidelines

/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(
/home/cytech/Documents/Data Battle/.venv/lib/python3.9/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


Thème sélectionné : Unity in PCT applications
Question générée : [{'QuestionNumber': 1, 'QuestionText': 'What is the purpose of amendments made during the PCT phase of a patent application?', 'Options': {'A': 'To ensure the application fulfills the requirements of Article 123(2) of the EPC.', 'B': 'To change the searched invention in another divisional application.', 'C': 'To pay additional search fees related to unity of invention.', 'D': 'To establish definitions in the description and drawings.'}, 'CorrectAnswer': 'A', 'Explanation': 'Amendments made during the PCT phase are primarily to ensure that the claims do not extend beyond the content of the originally filed application, as required by Article 123(2) of the EPC.', 'Question_type': 'MCQ', 'file': 'Generated Questions', 'Theme': 'Unity in PCT applications', 'Sources': {'original_filename': 'epo_guidelines_for_examination_2023_hyperlinked_en', 'title': 'international applications', 'section': '2 allowability amendments art 123 

In [16]:
import firebase_admin
from firebase_admin import credentials, firestore
import json

def ajouter_questions_firestore(chemin_fichier_json, nom_collection):
    if not firebase_admin._apps:
        cred = credentials.Certificate("data0battle-firebase-adminsdk-fbsvc-2fa20219b9.json")
        firebase_admin.initialize_app(cred)

    db = firestore.client()
    
    try:
        with open(chemin_fichier_json, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not isinstance(data, list):
            raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

        collection_ref = db.collection(nom_collection)

        # Récupérer le dernier identifiant utilisé pour l'incrémentation
        last_doc = collection_ref.order_by('question_id', direction=firestore.Query.DESCENDING).limit(1).stream()
        last_id = 0

        # Si des documents existent, récupérer le dernier identifiant
        for doc in last_doc:
            last_id = doc.to_dict().get('question_id', 0)
        
        # Incrémenter l'ID pour la prochaine question
        last_id += 1

        for question in data:
            if not isinstance(question, dict):
                print(f"⚠️ Erreur : Un élément du JSON n'est pas un dictionnaire : {question}")
                continue

            # Utiliser le dernier ID incrémenté
            question['question_id'] = last_id
            last_id += 1  # Incrémenter pour la prochaine question

            # Ajouter la question à Firestore avec l'identifiant unique incrémenté
            doc_ref = collection_ref.document(str(question['question_id']))
            doc_ref.set(question)

        print(f"✅ Données ajoutées avec succès à Firestore dans '{nom_collection}'.")

    except FileNotFoundError:
        print(f"❌ Le fichier '{chemin_fichier_json}' est introuvable, passage au fichier suivant.")
        return  # Continue sans arrêter le processus
    except json.JSONDecodeError:
        print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' n'est pas un JSON valide.")
    except ValueError as ve:
        print(f"❌ Erreur de format JSON : {ve}")
    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")


nom_collection = 'rag'

# Ajouter les questions de chaque fichier JSON correspondant à un thème
for theme in unique_themes:
    chemin_fichier_json = f'{theme}.json'
    ajouter_questions_firestore(chemin_fichier_json, nom_collection)


✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données ajoutées avec succès à Firestore dans 'rag'.
✅ Données 